In [5]:
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import polars as pl
import plotly.express as px

df = pl.read_parquet(
    "C:/Users/Arnold/OneDrive/Desktop/Information Visualization/Project/infoviz-project/data/processed/jobs_cleaned.parquet"
)

In [6]:

# Finding top 5 depts

top_depts = df.group_by("Business Title").agg(pl.col("# Of Positions").sum()).sort("# Of Positions",descending=True)

top_depts.head()


Business Title,# Of Positions
str,i64
"""Maintenance Worker""",405
"""Secetary 3A""",396
"""Housing Assistant""",396
"""City Seasonal Aide/Security Gu…",243
"""Child Protective Specialist""",150


In [7]:
## Finding the common skills in the dept

import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arnold\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
## Creating dictionary from the job description and the salary
# temp_df = df.select(
#     ((pl.col("Salary Range From")+ pl.col("Salary Range To"))/2).alias("Salary"),
#     "Minimum Qual Requirements"
# )
#
# temp_df["Min"]

In [10]:
# Dash app initialization
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Colors
colors = {
    'background': '#001F3F',
    'text': '#FFFFFF',
    'card': '#003366'
}

# Placeholder figures
fig = px.scatter_mapbox(
    df,
    lat="Latitude",
    lon="Longitude",
    color="Career Level",
    hover_name="Business Title",
    hover_data={"Latitude": False, "Longitude": False},
    zoom=10,
    height=500
)

fig_depts = px.bar(
    top_depts[:15],
    x="Business Title",
    y="# Of Positions"
)

# Styling updates
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=10,
    mapbox_center={"lat": 40.7128, "lon": -74.0060},
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

fig.update_traces(
    marker=dict(size=8, opacity=0.6),
    selector=dict(mode="markers"),
)

# Function to create cards
def create_card(title, value):
    return dbc.Card(
        dbc.CardBody([
            html.H5(title, className="card-title", style={'color': 'white', 'fontWeight': 'bold'}),
            html.H3(value, className="card-text", style={'color': 'white', 'fontWeight': 'bold'})
        ]),
        style={
            'backgroundColor': colors['card'],
            'borderRadius': '10px',
            'padding': '15px',
            'textAlign': 'center',
            'boxShadow': '2px 2px 10px rgba(0,0,0,0.3)',
            'width': '100%'
        }
    )

# Navbar
navbar = dbc.NavbarSimple(
    brand="NYC Federal Job Insights",
    brand_href="#",
    color="dark",
    dark=True
)

# Sidebar for filters
sidebar = dbc.Col([
    html.H4("Filter Jobs", className="text-light mt-3"),

    # Job Search Input
    dbc.InputGroup([
        dbc.Input(placeholder="Search by job title...", type="text"),
        dbc.Button("Search", color="primary")
    ], className="mb-3"),

    # Dropdown for Career Level
    html.Label("Career Level", className="text-light"),
    dcc.Dropdown(
        options=[
            {"label": "Entry Level", "value": "entry"},
            {"label": "Mid Level", "value": "mid"},
            {"label": "Senior Level", "value": "senior"}
        ],
        placeholder="Select level",
        className="mb-3"
    ),

    # Salary Range Slider
    html.Label("Salary Range", className="text-light"),
    dcc.RangeSlider(40000, 150000, step=5000, value=[50000, 100000],
                    marks={50000: "$50k", 100000: "$100k", 150000: "$150k"},
                    className="mb-3"),

    # Job Type Checkbox
    html.Label("Job Type", className="text-light"),
    dbc.Checklist(
        options=[
            {"label": "Full-time", "value": "full"},
            {"label": "Part-time", "value": "part"},
            {"label": "Remote", "value": "remote"}
        ],
        inline=True,
        className="mb-3"
    )
], width=3, className="bg-dark p-3 text-light")

# App layout
app.layout = dbc.Container([
    navbar,

    dbc.Row([
        sidebar,

        dbc.Col([
            # Job Statistics
            dbc.Row([
                dbc.Col(create_card("Total Jobs Posted", "2008"), width=4),
                dbc.Col(create_card("Top Hiring Dept.", "Dept. of Justice"), width=4),
                dbc.Col(create_card("Avg. Salary", "$82,000"), width=4)
            ], className="mb-4"),

            # Map and Histogram Side by Side
            dbc.Row([
                dbc.Col(dcc.Graph(figure=fig), width=6),
                dbc.Col(dcc.Graph(figure=fig_depts), width=6)
            ], className="mb-4"),

            # # Placeholder for Job Application Resources
            # html.Div([
            #     html.H4("Job Application Resources", className="text-light mb-3"),
            #     dbc.Row([
            #         dbc.Col(dbc.Button("Resume Tips", color="primary", className="w-100"), width=4),
            #         dbc.Col(dbc.Button("Interview Preparation", color="info", className="w-100"), width=4),
            #         dbc.Col(dbc.Button("Application Process", color="success", className="w-100"), width=4)
            #     ])
            # ], className="p-3 bg-secondary rounded")
        ], width=9)
    ], className="mt-3")
], fluid=True)

In [11]:

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)

In [34]:
df.columns

['Job ID',
 'Agency',
 '# Of Positions',
 'Business Title',
 'Civil Service Title',
 'Title Classification',
 'Title Code No',
 'Level',
 'Job Category',
 'Full-Time/Part-Time indicator',
 'Career Level',
 'Salary Range From',
 'Salary Range To',
 'Salary Frequency',
 'Work Location',
 'Division/Work Unit',
 'Job Description',
 'Minimum Qual Requirements',
 'Preferred Skills',
 'Additional Information',
 'To Apply',
 'Hours/Shift',
 'Work Location 1',
 'Recruitment Contact',
 'Residency Requirement',
 'Post Until',
 'Posting Updated',
 'Process Date',
 'Posting Date',
 'Posting Type',
 'Latitude',
 'Longitude']